In [1]:
import dask.array as da

from dask.distributed import Client, LocalCluster, wait

from lightgbm.dask import DaskLGBMRegressor

Create a cluster with 3 workers. Since this is a `LocalCluster`, those workers are just 3 local processes.

In [2]:
n_workers = 3
cluster = LocalCluster(
    n_workers=n_workers
)
client = Client(cluster)
client.wait_for_workers(n_workers)

print(f"View the dashboard: {cluster.dashboard_link}")

View the dashboard: http://127.0.0.1:8787/status


Click the link above to view a diagnostic dashboard while you run the training code below.

In [3]:
for i in range(10):
    print(f"attempt {i}")
    client.restart()

    num_rows = 1e6
    num_features = 1e2
    num_partitions = 10
    rows_per_chunk = num_rows / num_partitions

    data = da.random.random((num_rows, num_features), (rows_per_chunk, num_features))

    labels = da.random.random((num_rows, 1), (rows_per_chunk, 1))

    data = data.persist()
    labels = labels.persist()
    _ = wait(data)
    _ = wait(labels)

    dask_reg = DaskLGBMRegressor(
        silent=False,
        max_depth=5,
        random_state=708,
        objective="regression_l2",
        learning_rate=0.1,
        tree_learner="data",
        n_estimators=10,
        min_child_samples=1,
        n_jobs=-1,
        local_listen_port=12400,
    )

    dask_reg.fit(
        client=client,
        X=data,
        y=labels,
    )

attempt 0
worker_address_to_port
{'tcp://127.0.0.1:40729': 12400, 'tcp://127.0.0.1:34811': 12401, 'tcp://127.0.0.1:37625': 12402}
attempt 1
worker_address_to_port
{'tcp://127.0.0.1:40521': 12400, 'tcp://127.0.0.1:40601': 12401, 'tcp://127.0.0.1:39867': 12402}
attempt 2
worker_address_to_port
{'tcp://127.0.0.1:45167': 12400, 'tcp://127.0.0.1:37465': 12401, 'tcp://127.0.0.1:45003': 12402}
attempt 3
worker_address_to_port
{'tcp://127.0.0.1:39383': 12400, 'tcp://127.0.0.1:40419': 12403, 'tcp://127.0.0.1:38683': 12404}
attempt 4


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


worker_address_to_port
{'tcp://127.0.0.1:41139': 12400, 'tcp://127.0.0.1:37161': 12403, 'tcp://127.0.0.1:35383': 12405}
attempt 5


distributed.nanny - WARNING - Restarting worker


worker_address_to_port
{'tcp://127.0.0.1:32815': 12400, 'tcp://127.0.0.1:40245': 12405, 'tcp://127.0.0.1:46683': 12406}
attempt 6
worker_address_to_port
{'tcp://127.0.0.1:42911': 12400, 'tcp://127.0.0.1:43497': 12407, 'tcp://127.0.0.1:39533': 12408}
attempt 7
worker_address_to_port
{'tcp://127.0.0.1:34481': 12400, 'tcp://127.0.0.1:40305': 12407, 'tcp://127.0.0.1:35865': 12409}
attempt 8
worker_address_to_port
{'tcp://127.0.0.1:45223': 12400, 'tcp://127.0.0.1:39973': 12407, 'tcp://127.0.0.1:35969': 12410}
attempt 9
worker_address_to_port
{'tcp://127.0.0.1:40975': 12400, 'tcp://127.0.0.1:33181': 12401, 'tcp://127.0.0.1:38393': 12402}


In [ ]:
dask_reg.to_local().booster_.trees_to_dataframe()

In [ ]:
cluster.workers

In [ ]:
lines = open("/proc/net/tcp").readlines()
for line in lines:
    if "124" in line:
        print(line)

In [ ]:
import socket

location = ("127.0.0.1", 12402)
a_socket = socket.socket(
    socket.AF_INET,
    socket.SOCK_STREAM
)
result_of_check = a_socket.connect_ex(location)
print(result_of_check)
a_socket.close()

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ip = "127.0.0.1"
port = 12400
s.connect((ip, int(port)))
s.shutdown(2)

In [ ]:
import socket
import time

HOST = '127.0.0.1'
PORT = 12402

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)
    time.sleep(25)

In [ ]:
import socket
import time

HOST = '127.0.0.1'
PORT = 12402

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

## References

* https://realpython.com/python-sockets/
* http://www.serverframework.com/asynchronousevents/2011/01/time-wait-and-its-design-implications-for-protocols-and-scalable-servers.html